# Week 1 V3: PhysRobot with SV-Pipeline (Momentum-Conserving)

## What's new in V3
- **Unified PushBoxEnv**: single 16-dim obs environment (push_box_env.py retired)
- **PhysRobot uses SV-pipeline**: SVPhysicsCore guarantees momentum conservation by construction
- **Fixed training steps**: all 3 methods train for 500K steps for fair comparison
- **Clean code**: no emoji surrogate pairs, all code inline

## Architecture
```
PhysRobot V3 = Policy Stream (MLP) + Physics Stream (SV-GNN) + Fusion (concat+Linear)

SV-Pipeline: Scalarize -> MLP -> Vectorize
  For each undirected edge {i,j}:
    1. Build frame {e1, e2, e3} (antisymmetric)
    2. Project velocities onto frame -> rotation-invariant scalars
    3. MLP produces force coefficients alpha1, alpha2, alpha3
    4. Reconstruct: F_ij = alpha1*e1 + alpha2*e2 + alpha3*e3
    5. Assign +F to j, -F to i (Newton's 3rd law hard-coded)
  Result: sum_i F_i = 0 for ANY network parameters.
```

## Methods
1. **Pure PPO** -- standard MLP, no physics
2. **GNS + PPO** -- graph network, learns physics but no conservation guarantee
3. **PhysRobot (ours)** -- SV-pipeline dual-stream, momentum conservation by construction

In [ ]:
%%time
!pip install mujoco gymnasium stable-baselines3[extra] torch torch-geometric matplotlib pandas -q

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/physrobot_v3'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f'Save directory: {SAVE_DIR}')

In [ ]:
# ============================================================
# ENVIRONMENT: PushBoxEnv (Canonical 16-dim, unified)
# ============================================================
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces

XML = '''<mujoco model="push_box">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option timestep="0.002" integrator="Euler" gravity="0 0 -9.81">
    <flag warmstart="enable"/>
  </option>
  <visual>
    <global offwidth="1280" offheight="720"/>
    <quality shadowsize="4096"/>
    <map force="0.1" zfar="30"/>
  </visual>
  <asset>
    <texture builtin="gradient" height="100" rgb1="0.3 0.5 0.7" rgb2="0.1 0.2 0.3" type="skybox" width="100"/>
    <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
    <texture builtin="checker" height="100" name="texplane" rgb1="0.2 0.2 0.2" rgb2="0.3 0.3 0.3" type="2d" width="100"/>
    <material name="MatPlane" reflectance="0.3" shininess="0.5" specular="0.5" texrepeat="3 3" texture="texplane"/>
    <material name="geom" texture="texgeom" texuniform="true"/>
  </asset>
  <default>
    <joint armature="0.01" damping="0.1" limited="true"/>
    <geom conaffinity="1" condim="3" contype="1" friction="0.5 0.005 0.0001" margin="0.001" material="geom" rgba="0.8 0.6 0.4 1"/>
  </default>
  <worldbody>
    <light directional="true" diffuse="0.8 0.8 0.8" pos="0 0 3" dir="0 0 -1"/>
    <light directional="true" diffuse="0.4 0.4 0.4" pos="0 0 3" dir="1 1 -1"/>
    <geom name="floor" type="plane" size="3 3 0.1" rgba="0.8 0.8 0.8 1" material="MatPlane"/>
    <body name="arm_base" pos="0 0 0.02">
      <geom name="base_geom" type="cylinder" size="0.05 0.02" rgba="0.3 0.3 0.3 1"/>
      <body name="upper_arm" pos="0 0 0.02">
        <joint name="shoulder" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
        <geom name="upper_arm_geom" type="capsule" fromto="0 0 0 0.4 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
        <body name="forearm" pos="0.4 0 0">
          <joint name="elbow" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
          <geom name="forearm_geom" type="capsule" fromto="0 0 0 0.3 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
          <site name="endeffector" pos="0.3 0 0" size="0.03" rgba="1 0.5 0 0.8"/>
        </body>
      </body>
    </body>
    <body name="box" pos="0.35 0 0.05">
      <freejoint name="box_freejoint"/>
      <geom name="box_geom" type="box" size="0.05 0.05 0.05" mass="0.5" rgba="0.2 0.8 0.2 1" friction="0.5 0.005 0.0001"/>
      <site name="box_center" pos="0 0 0" size="0.01" rgba="0 1 0 1"/>
    </body>
    <site name="goal" pos="0.5 0.3 0.02" size="0.06" rgba="1 0 0 0.4" type="sphere"/>
  </worldbody>
  <actuator>
    <motor name="shoulder_motor" joint="shoulder" gear="1.0" ctrllimited="true" ctrlrange="-10 10"/>
    <motor name="elbow_motor" joint="elbow" gear="1.0" ctrllimited="true" ctrlrange="-10 10"/>
  </actuator>
</mujoco>
'''

class PushBoxEnv(gym.Env):
    """
    PushBox V3 (Canonical): 2-DOF arm pushes box to goal.
    Obs: 16-dim [joint_pos(2), joint_vel(2), ee_pos(3), box_pos(3), box_vel(3), goal(3)]
    Action: 2-dim joint torques in [-10, 10] Nm
    """
    def __init__(self, render_mode=None, box_mass=0.5):
        super().__init__()
        self.model = mujoco.MjModel.from_xml_string(XML)
        self.data = mujoco.MjData(self.model)
        self.box_mass = box_mass
        self._set_box_mass(box_mass)
        self._ee_site_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SITE, 'endeffector')
        self.action_space = spaces.Box(low=-10.0, high=10.0, shape=(2,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(16,), dtype=np.float32)
        self.goal_pos = np.array([0.5, 0.3, 0.02])
        self.max_episode_steps = 500
        self.current_step = 0
        self.success_threshold = 0.15
        self._prev_dist_box_goal = None
        self._prev_dist_ee_box = None
        self.render_mode = render_mode

    def _set_box_mass(self, mass):
        box_body_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, 'box')
        self.model.body_mass[box_body_id] = mass

    def set_box_mass(self, mass):
        self.box_mass = mass
        self._set_box_mass(mass)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)
        if seed is not None:
            np.random.seed(seed)
        self.data.qpos[0] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[1] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[2] = np.random.uniform(0.25, 0.45)
        self.data.qpos[3] = np.random.uniform(-0.15, 0.15)
        self.data.qpos[4] = 0.05
        self.data.qpos[5:9] = [1, 0, 0, 0]
        self.data.qvel[:] = 0.0
        mujoco.mj_forward(self.model, self.data)
        self.current_step = 0
        ee_pos = self.data.site_xpos[self._ee_site_id]
        box_pos = self.data.qpos[2:5]
        self._prev_dist_box_goal = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        self._prev_dist_ee_box = np.linalg.norm(ee_pos[:2] - box_pos[:2])
        return self._get_obs(), self._get_info()

    def _get_obs(self):
        joint_pos = self.data.qpos[:2].copy()
        joint_vel = self.data.qvel[:2].copy()
        ee_pos = self.data.site_xpos[self._ee_site_id].copy()
        box_pos = self.data.qpos[2:5].copy()
        box_vel = self.data.qvel[2:5].copy()
        return np.concatenate([joint_pos, joint_vel, ee_pos, box_pos, box_vel, self.goal_pos]).astype(np.float32)

    def _get_info(self):
        box_pos = self.data.qpos[2:5]
        d = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        return {'distance_to_goal': d, 'success': d < self.success_threshold,
                'box_mass': self.box_mass, 'timestep': self.current_step}

    def step(self, action):
        self.data.ctrl[:] = action
        for _ in range(5):
            mujoco.mj_step(self.model, self.data)
        ee_pos = self.data.site_xpos[self._ee_site_id].copy()
        box_pos = self.data.qpos[2:5].copy()
        dist_ee_box = np.linalg.norm(ee_pos[:2] - box_pos[:2])
        dist_box_goal = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        reach_progress = self._prev_dist_ee_box - dist_ee_box
        push_progress = self._prev_dist_box_goal - dist_box_goal
        self._prev_dist_ee_box = dist_ee_box
        self._prev_dist_box_goal = dist_box_goal
        reward = (
            0.5 * (-dist_ee_box) +
            1.0 * (-dist_box_goal) +
            10.0 * reach_progress +
            20.0 * push_progress +
            -0.01 * np.sum(action ** 2)
        )
        success = dist_box_goal < self.success_threshold
        if success:
            remaining = self.max_episode_steps - self.current_step
            reward += 500.0 + remaining * 1.0
        self.current_step += 1
        terminated = success
        truncated = self.current_step >= self.max_episode_steps
        return self._get_obs(), reward, terminated, truncated, self._get_info()

    def render(self): pass
    def close(self): pass

def make_push_box_env(box_mass=0.5):
    def _init():
        return PushBoxEnv(box_mass=box_mass)
    return _init

# Sanity check
env = PushBoxEnv()
obs, info = env.reset()
print(f'Obs shape: {obs.shape}, Action shape: {env.action_space.shape}')
print(f'Success threshold: {env.success_threshold}m')
env.close()
print('[OK] PushBoxEnv V3 loaded (unified 16-dim, progress reward)')

In [ ]:
# ============================================================
# SV MESSAGE PASSING (Physics Core -- pure PyTorch, no PyG)
# ============================================================
import torch
import torch.nn as nn
from typing import Tuple

EPS = 1e-7
DEG_EPS = 1e-4

def _make_mlp(in_dim, hidden_dim, out_dim):
    return nn.Sequential(
        nn.Linear(in_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.ReLU(),
        nn.Linear(hidden_dim, out_dim))

def build_edge_frames(pos, vel, src, dst):
    """Build antisymmetric edge frames. Returns e1, e2, e3, r_ij, d_ij."""
    r_ij = pos[dst] - pos[src]
    d_ij = torch.norm(r_ij, dim=-1, keepdim=True)
    e1 = r_ij / (d_ij + EPS)
    v_rel = vel[dst] - vel[src]
    v_par = (v_rel * e1).sum(dim=-1, keepdim=True) * e1
    v_perp = v_rel - v_par
    v_perp_norm = torch.norm(v_perp, dim=-1, keepdim=True)
    non_deg = (v_perp_norm > DEG_EPS).float()
    e2_vel = v_perp / (v_perp_norm + EPS)
    z_hat = torch.tensor([0., 0., 1.], device=pos.device).expand_as(e1)
    e2_f_raw = torch.cross(e1, z_hat, dim=-1)
    e2_f_norm = torch.norm(e2_f_raw, dim=-1, keepdim=True)
    use_y = (e2_f_norm < DEG_EPS).float()
    y_hat = torch.tensor([0., 1., 0.], device=pos.device).expand_as(e1)
    e2_f_raw2 = torch.cross(e1, y_hat, dim=-1)
    e2_f_raw = (1 - use_y) * e2_f_raw + use_y * e2_f_raw2
    e2_f_norm = torch.norm(e2_f_raw, dim=-1, keepdim=True)
    e2_fall = e2_f_raw / (e2_f_norm + EPS)
    e2 = non_deg * e2_vel + (1 - non_deg) * e2_fall
    e3 = torch.cross(e1, e2, dim=-1)
    return e1, e2, e3, r_ij, d_ij

class SVMessagePassing(nn.Module):
    """One round of SV message passing. Guarantees F_ij = -F_ji."""
    def __init__(self, node_dim, hidden_dim=32):
        super().__init__()
        n_scalar = 5  # ||r||, v_r, v_t, v_b, ||v_rel||
        self.force_mlp = _make_mlp(n_scalar + 2 * node_dim, hidden_dim, 3)
        self.node_update = _make_mlp(node_dim + 3, hidden_dim, node_dim)

    def _extract_pairs(self, edge_index):
        src, dst = edge_index[0], edge_index[1]
        mask = src < dst
        return torch.stack([src[mask], dst[mask]], dim=0)

    def forward_with_forces(self, h, edge_index, pos, vel):
        N = h.size(0)
        pairs = self._extract_pairs(edge_index)
        pi, pj = pairs[0], pairs[1]
        e1, e2, e3, r_ij, d_ij = build_edge_frames(pos, vel, pi, pj)
        v_rel = vel[pj] - vel[pi]
        v_r = (v_rel * e1).sum(-1, keepdim=True)
        v_t = (v_rel * e2).sum(-1, keepdim=True)
        v_b = (v_rel * e3).sum(-1, keepdim=True)
        v_norm = torch.norm(v_rel, dim=-1, keepdim=True)
        scalars = torch.cat([d_ij, v_r, v_t, v_b, v_norm,
                             h[pi] + h[pj], (h[pi] - h[pj]).abs()], dim=-1)
        alphas = self.force_mlp(scalars)
        force_ij = alphas[:, 0:1] * e1 + alphas[:, 1:2] * e2 + alphas[:, 2:3] * e3
        F_agg = torch.zeros(N, 3, device=h.device, dtype=h.dtype)
        F_agg.scatter_add_(0, pj.unsqueeze(-1).expand_as(force_ij), force_ij)
        F_agg.scatter_add_(0, pi.unsqueeze(-1).expand_as(force_ij), -force_ij)
        h_new = h + self.node_update(torch.cat([h, F_agg], dim=-1))
        return h_new, F_agg

class SVPhysicsCore(nn.Module):
    """Physics stream: node encoder + L x SV layers -> conserved forces."""
    def __init__(self, node_input_dim=6, hidden_dim=32, n_layers=1):
        super().__init__()
        self.encoder = _make_mlp(node_input_dim, hidden_dim, hidden_dim)
        self.sv_layers = nn.ModuleList([
            SVMessagePassing(node_dim=hidden_dim, hidden_dim=hidden_dim)
            for _ in range(n_layers)])

    def forward(self, positions, velocities, edge_index):
        h = self.encoder(torch.cat([positions, velocities], dim=-1))
        forces = None
        for layer in self.sv_layers:
            h, F_agg = layer.forward_with_forces(h, edge_index, positions, velocities)
            forces = F_agg
        return forces

# Quick conservation check
torch.manual_seed(42)
_m = SVPhysicsCore()
_p = torch.randn(4, 3); _v = torch.randn(4, 3)
_ei = torch.tensor([[0,0,0,1,1,1,2,2,2,3,3,3],
                     [1,2,3,0,2,3,0,1,3,0,1,2]], dtype=torch.long)
_F = _m(_p, _v, _ei)
print(f'Conservation check: ||sum F|| = {_F.sum(0).norm().item():.2e}')
assert _F.sum(0).norm().item() < 1e-4, 'CONSERVATION VIOLATED'
print('[OK] SVPhysicsCore loaded -- momentum conservation verified')

In [ ]:
# ============================================================
# AGENTS: Pure PPO, GNS, PhysRobot (SV-pipeline)
# ============================================================
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

try:
    from torch_geometric.nn import MessagePassing
    from torch_geometric.data import Data as PyGData, Batch as PyGBatch
    HAS_PYG = True
except ImportError:
    HAS_PYG = False
    print('torch_geometric not available -- GNS will use fallback MLP')

# ---------- Callback ----------
class SuccessTrackingCallback(BaseCallback):
    def __init__(self, verbose=1):
        super().__init__(verbose)
        self.episode_count = 0
        self.success_count = 0
        self.success_achieved = False
        self.episodes_to_success = None
    def _on_step(self):
        infos = self.locals.get('infos', [{}])
        dones = self.locals.get('dones', [False])
        for i, done in enumerate(dones):
            if done:
                self.episode_count += 1
                info = infos[i] if i < len(infos) else {}
                if info.get('success', False):
                    self.success_count += 1
                    if not self.success_achieved:
                        self.success_achieved = True
                        self.episodes_to_success = self.episode_count
                        print(f'\n** First success at episode {self.episode_count}! **')
                if self.episode_count % 200 == 0:
                    rate = self.success_count / self.episode_count * 100
                    print(f'  [Ep {self.episode_count}] Success rate: {rate:.1f}%')
        return True

# ---------- Agent 1: Pure PPO ----------
class PurePPOAgent:
    def __init__(self, env, verbose=1):
        self.model = PPO('MlpPolicy', env, learning_rate=3e-4, n_steps=2048,
                         batch_size=64, n_epochs=10, gamma=0.99, ent_coef=0.01,
                         verbose=verbose)
    def train(self, total_timesteps, callback=None):
        self.model.learn(total_timesteps=total_timesteps, callback=callback, progress_bar=True)
    def predict(self, obs, deterministic=True):
        action, _ = self.model.predict(obs, deterministic=deterministic)
        return action
    def save(self, path): self.model.save(path)
    def evaluate(self, env, n_episodes=100):
        rewards, successes = [], []
        for _ in range(n_episodes):
            obs, info = env.reset()
            done, ep_reward = False, 0
            while not done:
                action = self.predict(obs)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                done = terminated or truncated
            rewards.append(ep_reward)
            successes.append(1 if info.get('success', False) else 0)
        return {'mean_reward': np.mean(rewards), 'std_reward': np.std(rewards),
                'success_rate': np.mean(successes)}

# ---------- Agent 2: GNS (graph network, no conservation guarantee) ----------
if HAS_PYG:
    def obs_to_graph_batch(observations):
        B = observations.shape[0]
        dev = observations.device
        graphs = []
        for i in range(B):
            o = observations[i]
            ee_pos, box_pos = o[4:7], o[7:10]
            ee_vel = torch.zeros(3, device=dev)
            box_vel = o[10:13]
            node_feats = torch.stack([torch.cat([ee_pos, ee_vel]),
                                      torch.cat([box_pos, box_vel])])
            ei = torch.tensor([[0,1],[1,0]], dtype=torch.long, device=dev).t().contiguous()
            rel01 = box_pos - ee_pos
            rel10 = ee_pos - box_pos
            d01 = torch.norm(rel01).unsqueeze(0)
            d10 = torch.norm(rel10).unsqueeze(0)
            ea = torch.stack([torch.cat([rel01, d01]), torch.cat([rel10, d10])])
            graphs.append(PyGData(x=node_feats, edge_index=ei, edge_attr=ea))
        return PyGBatch.from_data_list(graphs)

    class GNSGraphLayerV2(MessagePassing):
        def __init__(self, node_dim, edge_dim, hidden_dim=32):
            super().__init__(aggr='add')
            self.edge_mlp = nn.Sequential(
                nn.Linear(2*node_dim + edge_dim, hidden_dim), nn.ReLU(),
                nn.Linear(hidden_dim, edge_dim))
            self.node_mlp = nn.Sequential(
                nn.Linear(node_dim + edge_dim, hidden_dim), nn.ReLU(),
                nn.Linear(hidden_dim, node_dim))
        def forward(self, x, edge_index, edge_attr):
            return self.propagate(edge_index, x=x, edge_attr=edge_attr)
        def message(self, x_i, x_j, edge_attr):
            return self.edge_mlp(torch.cat([x_i, x_j, edge_attr], dim=-1))
        def update(self, aggr_out, x):
            return self.node_mlp(torch.cat([x, aggr_out], dim=-1))

    class GNSFeaturesExtractorV2(BaseFeaturesExtractor):
        def __init__(self, observation_space, features_dim=64):
            super().__init__(observation_space, features_dim)
            hid, edge_dim = 32, 4
            self.node_encoder = nn.Sequential(nn.Linear(6, hid), nn.ReLU())
            self.edge_encoder = nn.Sequential(nn.Linear(edge_dim, hid), nn.ReLU())
            self.gn_layer = GNSGraphLayerV2(hid, hid, hid)
            self.decoder = nn.Linear(hid, 3)
            self.feature_proj = nn.Sequential(nn.Linear(3 + 16, features_dim), nn.ReLU())
        def forward(self, observations):
            graph = obs_to_graph_batch(observations)
            x = self.node_encoder(graph.x)
            ea = self.edge_encoder(graph.edge_attr)
            x = x + self.gn_layer(x, graph.edge_index, ea)
            acc = self.decoder(x)
            box_acc = acc[1::2]
            return self.feature_proj(torch.cat([box_acc, observations], dim=-1))

class GNSAgent:
    def __init__(self, env, verbose=1):
        if HAS_PYG:
            policy_kwargs = dict(
                features_extractor_class=GNSFeaturesExtractorV2,
                features_extractor_kwargs=dict(features_dim=64),
                net_arch=dict(pi=[64, 64], vf=[64, 64]))
        else:
            policy_kwargs = dict(net_arch=dict(pi=[64, 64], vf=[64, 64]))
        self.model = PPO('MlpPolicy', env, learning_rate=3e-4, n_steps=2048,
                         batch_size=64, n_epochs=10, gamma=0.99, ent_coef=0.01,
                         policy_kwargs=policy_kwargs, verbose=verbose)
    def train(self, total_timesteps, callback=None):
        self.model.learn(total_timesteps=total_timesteps, callback=callback, progress_bar=True)
    def predict(self, obs, deterministic=True):
        action, _ = self.model.predict(obs, deterministic=deterministic)
        return action
    def save(self, path): self.model.save(path)
    def evaluate(self, env, n_episodes=100):
        rewards, successes = [], []
        for _ in range(n_episodes):
            obs, info = env.reset()
            done, ep_reward = False, 0
            while not done:
                action = self.predict(obs)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                done = terminated or truncated
            rewards.append(ep_reward)
            successes.append(1 if info.get('success', False) else 0)
        return {'mean_reward': np.mean(rewards), 'std_reward': np.std(rewards),
                'success_rate': np.mean(successes)}

# ---------- Agent 3: PhysRobot V3 (SV-pipeline, conservation by construction) ----------
class PhysRobotFeaturesExtractorV3(BaseFeaturesExtractor):
    """
    Dual-stream extractor:
      Policy  = MLP(obs)         -> z_policy  [features_dim]
      Physics = SV-GNN(graph)    -> z_physics  [3]  (stop-gradient)
      Fusion  = concat + Linear  -> features   [features_dim]
    """
    def __init__(self, observation_space, features_dim=64):
        super().__init__(observation_space, features_dim)
        obs_dim = observation_space.shape[0]
        self.physics_core = SVPhysicsCore(node_input_dim=6, hidden_dim=32, n_layers=1)
        self.policy_stream = nn.Sequential(
            nn.Linear(obs_dim, 64), nn.ReLU(), nn.Linear(64, features_dim), nn.ReLU())
        self.fusion = nn.Sequential(
            nn.Linear(features_dim + 3, features_dim), nn.ReLU())
        self._edge_index_2 = torch.tensor([[0, 1], [1, 0]], dtype=torch.long).t()

    def _obs_to_graph(self, obs):
        ee_pos = obs[:, 4:7]
        box_pos = obs[:, 7:10]
        box_vel = obs[:, 10:13]
        ee_vel = torch.zeros_like(ee_pos)
        return ee_pos, ee_vel, box_pos, box_vel

    def forward(self, observations):
        B = observations.shape[0]
        device = observations.device
        z_policy = self.policy_stream(observations)
        ee_pos, ee_vel, box_pos, box_vel = self._obs_to_graph(observations)
        edge_index = self._edge_index_2.to(device)
        box_acc_list = []
        for i in range(B):
            pos_i = torch.stack([ee_pos[i], box_pos[i]], dim=0)
            vel_i = torch.stack([ee_vel[i], box_vel[i]], dim=0)
            acc_i = self.physics_core(pos_i, vel_i, edge_index)
            box_acc_list.append(acc_i[1])  # box node
        z_physics = torch.stack(box_acc_list, dim=0).detach()  # stop-gradient
        return self.fusion(torch.cat([z_policy, z_physics], dim=-1))

class PhysRobotAgent:
    def __init__(self, env, verbose=1):
        policy_kwargs = dict(
            features_extractor_class=PhysRobotFeaturesExtractorV3,
            features_extractor_kwargs=dict(features_dim=64),
            net_arch=dict(pi=[64, 64], vf=[64, 64]))
        self.model = PPO('MlpPolicy', env, learning_rate=3e-4, n_steps=2048,
                         batch_size=64, n_epochs=10, gamma=0.99, ent_coef=0.01,
                         policy_kwargs=policy_kwargs, verbose=verbose)
    def train(self, total_timesteps, callback=None):
        self.model.learn(total_timesteps=total_timesteps, callback=callback, progress_bar=True)
    def predict(self, obs, deterministic=True):
        action, _ = self.model.predict(obs, deterministic=deterministic)
        return action
    def save(self, path): self.model.save(path)
    def evaluate(self, env, n_episodes=100):
        rewards, successes = [], []
        for _ in range(n_episodes):
            obs, info = env.reset()
            done, ep_reward = False, 0
            while not done:
                action = self.predict(obs)
                obs, reward, terminated, truncated, info = env.step(action)
                ep_reward += reward
                done = terminated or truncated
            rewards.append(ep_reward)
            successes.append(1 if info.get('success', False) else 0)
        return {'mean_reward': np.mean(rewards), 'std_reward': np.std(rewards),
                'success_rate': np.mean(successes)}

# Print parameter counts
def count_params(model):
    return sum(p.numel() for p in model.parameters())

_tmp = DummyVecEnv([make_push_box_env()])
_a1 = PurePPOAgent(_tmp, verbose=0)
_a2 = GNSAgent(_tmp, verbose=0)
_a3 = PhysRobotAgent(_tmp, verbose=0)
print(f'Parameter counts:')
print(f'  Pure PPO:       {count_params(_a1.model.policy):>8,}')
print(f'  GNS V2:         {count_params(_a2.model.policy):>8,}')
print(f'  PhysRobot V3:   {count_params(_a3.model.policy):>8,}')
del _a1, _a2, _a3; _tmp.close()
print('[OK] All 3 agents loaded (PhysRobot V3 = SV-pipeline)')

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================
CONFIG = {
    'ppo_timesteps': 500_000,
    'gns_timesteps': 500_000,
    'physrobot_timesteps': 500_000,
    'n_envs': 4,
    'box_mass': 0.5,
    'eval_episodes': 100
}
print('V3 Configuration:', CONFIG)
est_min = 3 * CONFIG['ppo_timesteps'] / 200_000 * 5
print(f'Estimated training time: ~{est_min:.0f} min total on T4 GPU')

In [ ]:
%%time
# ============================================================
# TRAINING ALL 3 METHODS
# ============================================================
import time
import json
results = {}

def train_and_eval(name, AgentClass, timesteps_key):
    print('=' * 60)
    print(f'TRAINING {name} ({CONFIG[timesteps_key]:,} steps)')
    print('=' * 60)
    env = DummyVecEnv([make_push_box_env(CONFIG['box_mass']) for _ in range(CONFIG['n_envs'])])
    agent = AgentClass(env, verbose=0)
    callback = SuccessTrackingCallback(verbose=1)
    start = time.time()
    try:
        agent.train(CONFIG[timesteps_key], callback=callback)
        train_time = time.time() - start
        eval_env = PushBoxEnv(box_mass=CONFIG['box_mass'])
        eval_result = agent.evaluate(eval_env, n_episodes=CONFIG['eval_episodes'])
        eval_env.close()
        res = {
            'success_rate': eval_result['success_rate'],
            'mean_reward': eval_result['mean_reward'],
            'std_reward': eval_result['std_reward'],
            'episodes_to_success': callback.episodes_to_success,
            'total_episodes': callback.episode_count,
            'train_time_sec': train_time
        }
        agent.save(f'{SAVE_DIR}/{name.lower().replace(" ", "_")}')
        print(f'\n{name} results:')
        print(f'  Success rate: {res["success_rate"]:.1%}')
        print(f'  Mean reward:  {res["mean_reward"]:.1f}')
        print(f'  First success: ep {res["episodes_to_success"]}')
        print(f'  Training time: {train_time:.0f}s')
    except Exception as e:
        print(f'ERROR: {e}')
        res = {'error': str(e)}
    finally:
        env.close()
    return res

results['Pure PPO'] = train_and_eval('Pure PPO', PurePPOAgent, 'ppo_timesteps')
results['GNS'] = train_and_eval('GNS', GNSAgent, 'gns_timesteps')
results['PhysRobot V3'] = train_and_eval('PhysRobot V3', PhysRobotAgent, 'physrobot_timesteps')

# Save results
with open(f'{SAVE_DIR}/results_v3.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)
print('\nAll results saved.')

In [ ]:
# ============================================================
# RESULTS COMPARISON
# ============================================================
import pandas as pd
import matplotlib.pyplot as plt

rows = []
for method, res in results.items():
    if 'error' not in res:
        rows.append({
            'Method': method,
            'Success Rate': f"{res['success_rate']:.1%}",
            'Mean Reward': f"{res['mean_reward']:.1f} +/- {res['std_reward']:.1f}",
            'Ep to 1st Success': res['episodes_to_success'] or 'N/A',
            'Total Episodes': res['total_episodes'],
            'Train Time (s)': f"{res['train_time_sec']:.0f}"
        })

df = pd.DataFrame(rows)
print('\n=== Table 1: Sample Efficiency Comparison ===')
print(df.to_string(index=False))

# Bar chart
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

methods = [r['Method'] for r in rows]
success_rates = [results[m]['success_rate'] for m in results if 'error' not in results[m]]
ep_to_success = [results[m].get('episodes_to_success') or 0 for m in results if 'error' not in results[m]]

colors = ['#4C72B0', '#55A868', '#C44E52']

axes[0].bar(methods, success_rates, color=colors)
axes[0].set_ylabel('Success Rate')
axes[0].set_title('Final Success Rate')
axes[0].set_ylim(0, 1.05)

axes[1].bar(methods, ep_to_success, color=colors)
axes[1].set_ylabel('Episodes')
axes[1].set_title('Episodes to First Success')

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/results_v3.png', dpi=150)
plt.show()
print(f'Figure saved to {SAVE_DIR}/results_v3.png')

In [ ]:
# ============================================================
# OOD GENERALIZATION TEST (Table 2)
# ============================================================
print('=== OOD Generalization Test ===')
print('Testing with box masses: 0.5, 1.0, 2.0, 5.0 kg')

ood_masses = [0.5, 1.0, 2.0, 5.0]
ood_results = {}

# Reload best agents
agents = {
    'Pure PPO': PurePPOAgent,
    'GNS': GNSAgent,
    'PhysRobot V3': PhysRobotAgent
}

for method_name, AgentClass in agents.items():
    ood_results[method_name] = {}
    # Create a dummy env to load the agent
    dummy_env = DummyVecEnv([make_push_box_env()])
    agent = AgentClass(dummy_env, verbose=0)
    model_path = f'{SAVE_DIR}/{method_name.lower().replace(" ", "_")}'
    try:
        agent.model = PPO.load(model_path)
    except Exception:
        print(f'  Could not load {model_path}, skipping...')
        dummy_env.close()
        continue
    dummy_env.close()

    for mass in ood_masses:
        test_env = PushBoxEnv(box_mass=mass)
        res = agent.evaluate(test_env, n_episodes=50)
        ood_results[method_name][mass] = res['success_rate']
        test_env.close()
        print(f'  {method_name} @ mass={mass}: success={res["success_rate"]:.1%}')

# Plot OOD results
fig, ax = plt.subplots(figsize=(8, 5))
for i, (method, mass_results) in enumerate(ood_results.items()):
    if mass_results:
        masses = sorted(mass_results.keys())
        rates = [mass_results[m] for m in masses]
        ax.plot(masses, rates, 'o-', label=method, color=colors[i], linewidth=2)

ax.set_xlabel('Box Mass (kg)')
ax.set_ylabel('Success Rate')
ax.set_title('OOD Generalization (train mass=0.5, test on different masses)')
ax.legend()
ax.set_ylim(-0.05, 1.05)
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/ood_v3.png', dpi=150)
plt.show()
print(f'OOD figure saved to {SAVE_DIR}/ood_v3.png')

In [ ]:
# ============================================================
# CONSERVATION VERIFICATION (PhysRobot only)
# ============================================================
print('=== Momentum Conservation Verification ===')
print('Running 100 random trials across different graph sizes...\n')

# Test with the standalone SVPhysicsCore
test_model = SVPhysicsCore(node_input_dim=6, hidden_dim=32, n_layers=1)
all_errors = []

for N in [2, 3, 4, 5, 8]:
    errors = []
    for _ in range(100):
        pos = torch.randn(N, 3)
        vel = torch.randn(N, 3)
        src, dst = [], []
        for i in range(N):
            for j in range(N):
                if i != j:
                    src.append(i); dst.append(j)
        ei = torch.tensor([src, dst], dtype=torch.long)
        with torch.no_grad():
            F = test_model(pos, vel, ei)
        err = F.sum(0).norm().item()
        errors.append(err)
    max_err = max(errors)
    all_errors.extend(errors)
    status = 'PASS' if max_err < 1e-3 else 'FAIL'
    print(f'  N={N}: max ||sum F|| = {max_err:.2e}  [{status}]')

print(f'\nOverall: max error across all trials = {max(all_errors):.2e}')
print(f'Conservation guaranteed by architecture (not learned).')

In [ ]:
# ============================================================
# SUMMARY
# ============================================================
print('=' * 60)
print('V3 EXPERIMENT COMPLETE')
print('=' * 60)
print()
print('Key results:')
for method, res in results.items():
    if 'error' not in res:
        print(f'  {method}:')
        print(f'    Success rate:       {res["success_rate"]:.1%}')
        print(f'    Ep to 1st success:  {res["episodes_to_success"]}')
        print(f'    Mean reward:        {res["mean_reward"]:.1f}')
print()
print('What V3 proves:')
print('  1. SV-pipeline PhysRobot maintains momentum conservation by construction')
print('  2. All methods trained for same number of steps (fair comparison)')
print('  3. Physics stream predicts forces, not just embeddings')
print()
print(f'All outputs saved to: {SAVE_DIR}')